In [2]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

## Auto Summarizing Text

Problem Definition  
Retrieve Text  (bs4)  
Pre-processing  
Visualisation  
Extract Sentences

### Libraries

In [2]:
from bs4 import BeautifulSoup
import requests
from nltk import tokenize
from nltk.corpus import stopwords 
from nltk.tokenize import sent_tokenize, word_tokenize
from string import punctuation
import heapq
from nltk.stem.porter import *

### Retrieve Text

In [3]:
url = 'https://www.scmp.com/news/hong-kong/politics/article/2176058/hong-kong-lawmaker-eddie-chus-ban-village-election-based'
def GetText(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser')
    text = ' '.join(map(lambda p: p.text, soup.find_all('p')))
    return text

In [4]:
GetText(url)

" Hong Kong localism and independence Comments: \xa0  The decision to ban lawmaker Eddie Chu Hoi-dick from running in a rural representative election was based on a shaky argument that could be struck down in court, according to leading legal scholars, who also called on Hong Kong’s courts to clarify the vagueness in election laws. Johannes Chan Man-mun, the former law dean of the University of Hong Kong, was speaking on Sunday after Chu was told he would not be allowed to run for a post as a local village’s representative. Returning officer Enoch Yuen Ka-lok pointed to Chu’s stance on Hong Kong independence and said the lawmaker had dodged his questions on his political beliefs. Yuen took this to imply that Chu supported the possibility of Hong Kong breaking with Beijing in the future. Chan, however, said Chu’s responses to the returning officer were open to interpretation. The legal scholar did not believe they met the standard of giving the election officer “cogent, clear and compel

### Pre-processing

In [5]:
def preprocess(word_token):
    stop_words = set(stopwords.words('english')+list(punctuation)+['’','“','–','”',',','.','‘'])
    word_frequencies = {}
    for word in word_token:
        if word not in stop_words:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1
    return word_frequencies    

In [6]:
sentences = sent_tokenize(GetText(url))[0:-2]
word_token = word_tokenize(GetText(url))
word_frequencies = preprocess(word_token)

In [7]:
sum(word_frequencies.values())

625

In [8]:
def frequency(word_frequencies):
    total_count = sum(word_frequencies.values())
    for word in word_frequencies.keys():  
        word_frequencies[word] = (word_frequencies[word]/total_count) # TF
    return word_frequencies

In [9]:
word_frequencies = frequency(word_frequencies)
word_frequencies

{"'d": 0.0016,
 "'s": 0.0016,
 '2003': 0.0016,
 '2016': 0.0032,
 '24': 0.0016,
 'A': 0.0016,
 'According': 0.0016,
 'Additional': 0.0016,
 'Administrative': 0.0016,
 'Andy': 0.0064,
 'Aside': 0.0016,
 'At': 0.0016,
 'Basic': 0.0112,
 'Beijing': 0.0016,
 'Both': 0.0016,
 'But': 0.0016,
 'Chan': 0.0176,
 'Cheung': 0.0032,
 'Chu': 0.04,
 'Comments': 0.0016,
 'Council': 0.0048,
 'Eddie': 0.0016,
 'Election': 0.0016,
 'Enoch': 0.0016,
 'Eric': 0.0016,
 'Even': 0.0016,
 'Executive': 0.0016,
 'February': 0.0016,
 'Fisher': 0.0064,
 'Gladys': 0.0016,
 'HKU': 0.0016,
 'He': 0.0032,
 'Ho-tin': 0.0016,
 'Hoi-dick': 0.0016,
 'Hong': 0.0176,
 'Hongkongers': 0.0016,
 'It': 0.0016,
 'January': 0.0016,
 'Johannes': 0.0048,
 'Ka-lok': 0.0016,
 'Ka-wah': 0.0016,
 'Kong': 0.0176,
 'Law': 0.0112,
 'Legco': 0.0032,
 'Legislative': 0.0048,
 'Li': 0.0048,
 'Lok-kei': 0.0016,
 'Man-mun': 0.0016,
 'Monday': 0.0016,
 'New': 0.0016,
 'Ordinance': 0.0016,
 'Pressed': 0.0016,
 'Region': 0.0016,
 'Representative': 

In [10]:
def sent_score(sentences):
    sentence_scores = {}
    for sentence in sentences:
        for word in word_tokenize(sentence.lower()):
            if word in word_frequencies.keys():
                if len(sentence.split(' ')) < 30:
                    if sentence not in sentence_scores.keys():
                        sentence_scores[sentence] = word_frequencies[word]
                    else: 
                        sentence_scores[sentence] += word_frequencies[word]
    return sentence_scores

In [11]:
sentence_scores = sent_score(sentences)

In [12]:
summary_sentences = heapq.nlargest(3, sentence_scores, key=sentence_scores.get)
summary = ' '.join(summary_sentences)  
print(summary)  

Johannes Chan said Chu’s political stance was open to interpretation, and the election officer could hardly fulfil the criteria for providing “cogent, clear and compelling” evidence to disqualify him. Returning officer Enoch Yuen Ka-lok pointed to Chu’s stance on Hong Kong independence and said the lawmaker had dodged his questions on his political beliefs. Tong also said it might not have made a difference, had Chu answered all the questions raised by the returning officer.
